In [1]:
pip install mediapipe

     |████████████████████████████████| 47.4MB 66kB/s 


In [7]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline


def get_actual_angle(x1, y1, x2, y2):
  return math.atan((y2-y1)/(x2-x1)) * 180/ math.pi

def get_angle(x1, y1, x2, y2): 
  return math.atan((y2-y1)/(x2-x1)) * 180/ math.pi

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
count = 1
plt.figure(figsize=(6,6))
skeletons = []
# For static images:
file_list = ['AmitHands.jpeg','SagarHands.jpeg']
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
  for idx, file in enumerate(file_list):
    image = cv2.imread(file)
    image_height, image_width, _ = image.shape
    # Convert the BGR image to RGB before processing.
    print(image.shape)
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if not results.pose_landmarks:
      continue

    
    '''print(
        f'Nose coordinates: ('
        f'{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width}, '
        f'{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y * image_height})'
    )'''
    
    # Draw pose landmarks on the image.
    annotated_image = image.copy()
    
    # Use mp_pose.UPPER_BODY_POSE_CONNECTIONS for drawing below when
    # upper_body_only is set to True.
    mp_drawing.draw_landmarks(
        annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    dark_canvas = np.zeros(image.shape, np.uint8)
    mp_drawing.draw_landmarks(
      dark_canvas, results.pose_landmarks, mp_pose.POSE_CONNECTIONS 
    )
    cv2.imwrite('skeleton_image' + str(idx) + '.png', dark_canvas)
    cv2.imwrite('annotated_image' + str(idx) + '.png', annotated_image)

    annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
    plt.subplot(2,2,count)
    plt.imshow(annotated_image)
    plt.title(file)
    plt.yticks([])
    plt.xticks([])

    plt.subplot(2,2,count+1)
    plt.imshow(dark_canvas)
    plt.title(file)
    plt.yticks([])
    plt.xticks([])

    count += 2

    skeletons.append(results)
'''
for index in range(32):
  for result in skeletons:
    print(result.pose_landmarks.landmark[index].x * image_width, result.pose_landmarks.landmark[index].y * image_height)
    
  print("====")'''
angles_images = []
edges = [(14,16), (12,14), (11,12), (13,11), (15, 13), (24,12), (23,11), (23,24), (26,24), (25,23), (28,26), (27,25)]
actual_angles = []
visibilities = []
for result in skeletons:
  angles_image = []
  actual_angle = []
  visibility = []
  '''print(result.pose_landmarks.landmark[14].x  * image_width, 
        result.pose_landmarks.landmark[14].y * image_height, 
        result.pose_landmarks.landmark[16].x * image_width, 
        result.pose_landmarks.landmark[16].y * image_height )

  print(result.pose_landmarks.landmark[13].x  * image_width, 
        result.pose_landmarks.landmark[13].y * image_height, 
        result.pose_landmarks.landmark[15].x * image_width, 
        result.pose_landmarks.landmark[15].y * image_height)'''
  for edge in edges:
    start, dest = edge
  
    angles_image.append(get_angle(result.pose_landmarks.landmark[start].x  * image_width, 
                                  result.pose_landmarks.landmark[start].y * image_height, 
                                  result.pose_landmarks.landmark[dest].x * image_width, 
                                  result.pose_landmarks.landmark[dest].y * image_height ))    
    visibility.append(result.pose_landmarks.landmark[start].visibility+result.pose_landmarks.landmark[dest].visibility)
  angles_images.append(angles_image)
  visibilities.append(visibility)

'''for angles_image in angles_images:
  print(angles_image)
'''
difference = [0 for i in range(len(edges))]
for i in range(len(edges)):
  difference[i] = abs(angles_images[1][i] - angles_images[0][i])
  if angles_images[1][i] < 0:
      difference[i] = min(abs(180 - abs(angles_images[1][i]) - angles_images[0][i]), difference[i])
  elif angles_images[0][i] < 0:
      difference[i] = min(abs(180 - abs(angles_images[0][i]) - angles_images[1][i]), difference[i])
      
match_percent = [(90-difference[i])/90 * 100 for i in range(len(edges))]
weighted_mask = [8, 4, 1, 4, 8, 1, 1, 1, 4, 4, 8, 8]
final_percentage_each = [match_percent[i] * weighted_mask[i]/sum(weighted_mask) 
                        for i in range(len(edges))]

print(angles_images[0])
print(angles_images[1])
print(difference)
print(f"MATCH PERCENTAGE: {round(sum(final_percentage_each),2)}%")

#print(difference)
#print(values)
plt.show()

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic

video_file = 'AmitHands.mp4'
from google.colab.patches import cv2_imshow

cap = cv2.VideoCapture(video_file)
count = 0
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:

      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      break
      #continue
    count += 1
    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2_imshow(image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
print(count)
cap.release()